In [92]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from PyNomaly import loop


import os
import pandas as pd
from pathlib import Path

V_TO_MV = 1000
V_TO_GRAM = -25000
lop_score_threshold = 0.2

def main():
   
    pd.set_option('display.max_rows', 100)
    unfiltered_df = get_multiIndex(Path(r'C:\Users\asenn\OneDrive\School\Research\MSIPP (Georgia 2023)\Automated Data'))
    filtered_df = pd.DataFrame(index=unfiltered_df.index, columns=unfiltered_df.columns)
    filtered_df.loc[('T1'), 'trial_data'][0:2] = unfiltered_df.loc[('T1'), 'trial_data'][0:2].apply(lambda x: filter_and_trim_data(x, trim_start=300, trim_end=200))  # type: ignore
    display(filtered_df)
    display(filtered_df.loc[('T1'), 'trial_data'][0:2][0]) # type: ignore


    # display((unfiltered_df.loc[('T1', 'E1'), 'trial_data']))
    # unfiltered_df[:2].loc['T1', 'trial_data'] = unfiltered_df[:2].loc['T1', 'trial_data'].apply(lambda x: filter_data(x))
    # display((unfiltered_df.loc[('T1', 'E1'), 'trial_data']))

    

    # display(unfiltered_df['trial_data'])
    # unfiltered_df[:2]['trial_data'] = unfiltered_df[:2]['trial_data'].apply(lambda x: filter_data(x))
    # display(unfiltered_df[:2]['trial_data'])


    

def get_multiIndex(folder: Path) -> pd.DataFrame:
    """Put data into MultiIndex DataFrame"""

    trials: list[pd.DataFrame] = []
    multiindex_tuples: list[tuple[str, str]] = []

    for file in os.listdir(folder):
            temperature, trial_index = get_index(Path(os.path.join(folder, file))) # parse MultiIndex labels
            trial = pd.read_csv(os.path.join(folder, file)) # Read data
            trial.drop(trial.columns[2], axis='columns', inplace=True) # drop second time index
            trial.iloc[:,2] =  trial.iloc[:,2] * V_TO_GRAM
            trial.iloc[:,1]  = trial.iloc[:,1] * V_TO_MV
            trial.columns = ['Time (s)', 'Response (mV)', 'Load (g)']
            trial.set_index(keys= 'Time (s)', inplace=True)
            trials.append(trial)
            multiindex_tuples.append((temperature, trial_index))
     
    multiindex=pd.MultiIndex.from_tuples(multiindex_tuples, names=['Temperature', 'Experiment'])
    unfiltered_data = pd.DataFrame({'trial_data':trials}, index=multiindex)
    return unfiltered_data



def filter_and_trim_data(unfiltered_df: pd.DataFrame, trim_start: int, trim_end: int) -> pd.DataFrame:
    """Filters input DataFrame for noise using LOP. Returns filtered DataFrame."""
    unfiltered_df = unfiltered_df.iloc[trim_start: (len(unfiltered_df) - trim_end)]
    m = loop.LocalOutlierProbability(unfiltered_df.drop(columns='Load (g)'), extent=.25, n_neighbors = 20).fit() # Used to find scores
    scores = m.local_outlier_probabilities # this is an attribute after performing LocalOutlierProbability()
    unfiltered_df['scores'] = scores.tolist() # Adding scores to lop_df
    return unfiltered_df.query(f'scores < {lop_score_threshold}')

def get_index(filepath: Path) -> tuple[str, str]:
    """Parse trial filenames and get the temperature and trial."""
    temperature = filepath.stem[:2]
    trial_index = filepath.stem[2:]
    return temperature, trial_index

if __name__ == "__main__":
    main()

c:\Users\asenn\pycode\pyprojects\pvc-gels\.venv.pvc-gels\Lib\site-packages\PyNomaly\loop.py:332: UserWarning: Argument 'extent' is not of type (<class 'int'>, <class 'numpy.integer'>).
  warnings.warn("Argument %r is not of type %s." % (
c:\Users\asenn\pycode\pyprojects\pvc-gels\.venv.pvc-gels\Lib\site-packages\PyNomaly\loop.py:224: UserWarning: extent parameter (lambda) must be 1, 2, or 3.
  warnings.warn(
C:\Users\asenn\AppData\Local\Temp\ipykernel_6684\1538183346.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unfiltered_df['scores'] = scores.tolist() # Adding scores to lop_df
c:\Users\asenn\pycode\pyprojects\pvc-gels\.venv.pvc-gels\Lib\site-packages\PyNomaly\loop.py:332: UserWarning: Argument 'extent' is not of type (<class 'int'>, <clas

trial_data
Temperature Experiment                                                   
T1          E1                    Response (mV)    Load (g)    scores
...
            E10                   Response (mV)    Load (g)    scores
...
            E100                                                      NaN
            E11                                                       NaN
            E12                                                       NaN
...                                                                   ...
T3          E95                                                       NaN
            E96                                                       NaN
            E97                                                       NaN
            E98                                                       NaN
            E99                                                       NaN

[300 rows x 1 columns]

,Response (mV),Load (g),scores
Time (s),,,
20.39885,6.122581,48.174875,0.000000
20.53220,6.194233,48.218175,0.098197
20.59887,6.155068,48.196325,0.000000
20.73223,6.102990,48.106850,0.000000
20.79890,6.103139,48.133650,0.000000
...,...,...,...
37.40001,9.484016,154.599300,0.000000
37.53335,9.489655,154.418300,0.000000
37.60001,9.496384,154.483025,0.000000


In [ ]:
file = 'T1E12.csv'
#! C:\Users\asenn\OneDrive\School\Research\MSIPP (Georgia 2023)\Automated Data
folder = Path(r'C:\Users\asenn\OneDrive\School\Research\MSIPP (Georgia 2023)\Automated Data')
trial = pd.read_csv(os.path.join(folder, file))
print(trial.iloc[:,1])



#!########## 
m = loop.LocalOutlierProbability(lop_df.drop(columns='Load (g)'), extent=.25, n_neighbors = 20).fit() # Used to find scores
scores = m.local_outlier_probabilities # this is an attribute after performing LocalOutlierProbability()
lop_df['scores'] = scores.tolist() # Adding scores to lop_df
filtered_df = lop_df.query(f'scores < {lop_score_threshold}') # Final df with dropped outliers

#!#######